In [43]:
from tkinter import *
import tkinter as tk
from PIL import ImageTk, Image

# Import the mark_attendance function here
from PIL import Image
import numpy as np
import cv2
import pickle
from tensorflow.keras.models import load_model

import datetime
import mysql.connector

def mark_attendance():

    # for face detection
    face_cascade =cv2.CascadeClassifier(
        'haarcascade_frontalface_default.xml')

    # resolution of the webcam
    screen_width = 1280       # try 640 if code fails
    screen_height = 720

    # size of the image to predict
    image_width = 224
    image_height = 224

    # load the trained model
    model = load_model('transfer_learning_trained_face_cnn_model.h5')

    # the labels for the trained model
    with open("face-labels.pickle", 'rb') as f:
        og_labels = pickle.load(f)
        labels = {key:value for key,value in og_labels.items()}
        print(labels)
        
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="navaneetha",
        database="faceAtt"
    )
    
    cursor = db.cursor()

    # default webcam
    stream = cv2.VideoCapture(0)
    flag=1
    result=""

    while(flag):
        # Capture frame-by-frame
        (grabbed, frame) = stream.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # try to detect faces in the webcam
        faces = face_cascade.detectMultiScale(
            rgb, scaleFactor=1.3, minNeighbors=5)

        # for each faces found
        for (x, y, w, h) in faces: 
            roi_rgb = rgb[y:y+h, x:x+w]

            # Draw a rectangle around the face
            color = (255, 0, 0) # in BGR
            stroke = 2
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, stroke)

            # resize the image
            size = (image_width, image_height)
            resized_image = cv2.resize(roi_rgb, size)
            image_array = np.array(resized_image, "uint8")
            img = image_array.reshape(1,image_width,image_height,3) 
            img = img.astype('float32')
            img /= 255

            # predict the image
            predicted_prob = model.predict(img,verbose=0)

            # Display the label
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[predicted_prob[0].argmax()]
            color = (255, 0, 255)
            stroke = 2
            cv2.putText(frame, f'({name})', (x,y-8), font, 1, color, stroke, cv2.LINE_AA)

            now=datetime.datetime.now()
            d = now.date()
            t = now.time()
            cur = cursor.execute(f"SELECT * FROM attendance WHERE username='{name}' AND attDate='{d}'")
            
            if len(cursor.fetchall()) > 0:
                result= (f"{name} has already attended today.")
                flag=0
            else:
                cv2.imshow("Image", frame)
                key = cv2.waitKey(1) & 0xFF

                if key == 13:
                    # add a new record to the database
                    sql = "INSERT INTO attendance (username,attDate,attTime) VALUES (%s,%s,%s)"
                    val = (name,d,t)
                    cursor.execute(sql, val)
                    result= (f"{name} has been marked as attended on {d}.")
                    db.commit()
                    flag=0
                

        # show the frame   
        cv2.imshow("Image", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:    # Press q to break out of the loop
            break      

    # Cleanup
    stream.release()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    return result

class AttendanceApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Attendance App")

        # Create a label to display the webcam stream
        self.image_label = Label(self.master)
        self.image_label.pack()

        # Create a button to mark attendance
        self.attendance_button = Button(self.master, text="Mark Attendance", command=self.mark_attendance,width=20,height=3)
        self.attendance_button.pack()

        # Create a label to display the status message
        self.status_label = Label(self.master, text="")
        self.status_label.pack()

        # Create a quit button
        self.quit_button = Button(self.master, text="Quit", command=self.quit,width=20,height=3)
        self.quit_button.pack()

    def mark_attendance(self):
        # Call the mark_attendance function here
        # Update the status label with the result
        
        result = mark_attendance()
        self.status_label.config(text=result)

    def quit(self):
        self.master.destroy()

if __name__ == "__main__":
    # Create the tkinter window
    root = Tk()
    root.geometry("400x400")
    root.configure(bg='skyblue')


    # Create the AttendanceApp object
    app = AttendanceApp(root)

    # Start the tkinter event loop
    root.mainloop()
